In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# import data from C:\Data\hershey_promo_impact\shipment_hist_weekly_with_promo_features.csv
data = pd.read_csv("C:\\Data\\hershey_promo_impact\\shipment_hist_weekly_with_promo_features.csv")

# check data
data.head()



,week_start_date,sales_org,dp_cust,from_loc,dmd_item_10,qty,descr,division,franchise,subbrand,...,promotion_type,total_expense_trade,forecast_cot_spend,price_sch3,po_qty_week_1,po_qty_week_2,po_qty_week_3,po_qty_week_4,po_qty_week_8,avg_order_priority
0,2021-09-13,US04,6902037,5103,77800819,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-09-13,US04,6902037,5208,1566500023,0.0,Sam H PirBty AgWtChdr Vbx 40pk 20oz,Salty Snacking,PIRATES BOOTY,PIRATE BRANDS PB BOOTY,...,EDLP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-09-13,US04,6902037,5208,1692502250,0.0,SP HotCocoa 17oz,Salty Snacking,SKINNYPOP,SKINNYPOP RTE,...,EDLP,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-09-13,US04,6902038,5103,5571200802,96.0,DtPtz Org 1.5oz 60,Salty Snacking,DOTS,DOTS NO SUB,...,Correction,-332.91,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-09-13,US04,6902038,5401,1566500030,0.0,V PirBty AgWtChdr 12pk 6oz 12,Salty Snacking,PIRATES BOOTY,PIRATE BRANDS PB BOOTY,...,Hi Lo,7121.11,352076.72,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# check unique values for combination of week_st